First attempt at code. Started second cell because of this error.

In [37]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')

import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:\Users\kylek_000\Documents\geog458 digital\Lab data"

in_state = "saep_bg10.shp"                                                    #State input shapefile
in_table = "WashingtonFIPS.dbf"                                               #State input table

#list of counties, only top 10 populated to be printed counties[0:10] IF ordered by highest to lowest population
counties = []

def county_sep(state, table):
    
    #Adding Total_POP field in county table in order to print out the most populated counties later
    arcpy.AddField_management(table, "Total_POP", "FLOAT")
    
    with arcpy.da.UpdateCursor(table, ["FIPSCounty"]) as cursor:                        #create cursor for county table
        
        
        for row in cursor:                                                              #inside county table
            population = 0.0                                                            #population count
            with arcpy.da.UpdateCursor(state, ["COUNTYPF10", "POP2013"]) as cursor2:    #create cursor for block group table
                
                #creates the file for the county
                county = arcpy.CreateFeatureclass_management("C:\Users\kylek_000\Documents\geog458 digital\Lab data", str(row[0]))
                
                #creating field in county shapefile for population
                arcpy.AddField_management(county, "TOTALPOP")
                
                #create UPDATE cursor for file in order to add column and data
                update = arcpy.da.UpdateCursor(county, ["TOTALPOP"]) 
                
                for data in cursor2:                                                    #inside block group table
                    if row[0] == data[0]:
                        population += data[1]
                        #ALSO NEED TO ADD SHAPES INTO THE COUNTIES IN THIS if statement
                        
                        update.updateRow(population)                    
                        cursor.updateRow(population)
                del update

    
    #function to sort the list by population
    
    #print counties[0:10]
    

county_sep(in_state, in_table)


RuntimeError: A column was specified that does not exist.

Second attempt. I got the files made and made them into geojson files.
I changed some incorrect indexes and added the spatial projections because the geojson files weren't visually showing up.
Got this error again. I spent a good +10 hours of coding and trying to figure all this out. This error is killing me. 
My parameters seem to be fine, and I created made some checks along the way. Nobody could help. 

In [155]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')

import arcpy

from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"


arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:\Users\kylek_000\Documents\geog458 digital\Lab data"

in_state = "saep_bg10.shp"                                                     #State input shapefile
in_counties = "WashingtonFIPS.dbf"                                             #Counties table

def county_sep(state, table):
    
    arcpy.AddField_management(table, "Total_POP", "FLOAT")
    projection = arcpy.Describe(state).spatialReference
    
    with arcpy.da.UpdateCursor(table, ["FIPSCounty", "CountyName"]) as cursor:                 #create cursor traversing county table 
        for row in cursor:                                                                     #traverse county table
            
            population = 0.0
            
            #create shapefile
            county = arcpy.CreateFeatureclass_management ("C:\\Users\\kylek_000\\Documents\\geog458 digital\\Lab data\\outputs\\", str(row[1]))
            #arcpy.AddField_management(county, "FIPSCounty", "LONG")
            #arcpy.AddField_management(county, "Total_Pop", "FLOAT")
            
            with arcpy.da.SearchCursor(state, ["COUNTYFP10", "Total_POP"]) as cursor2:
                
                with arcpy.da.InsertCursor(county, ["COUNTYFP10", "Total_POP"]) as cursor3: 
                    for row2 in cursor2:
                        
                        #comparing FIPS codes 
                        if row[0] == row2[0]:
                            #print str(row2[1])
                            cursor3.insertRow(row2)
                            population += row2[1]
                            cursor.updateRow(population)
                    
            #adds projection to shapefiles
            arcpy.management.DefineProjection(county, projection)
            call(['C:\\OSGeo4W\\bin\\ogr2ogr','-f','GeoJSON', '-t_srs','WGS84', '-s_srs','EPSG:2927', 'C:\\Users\\kylek_000\\Documents\\geog458 digital\\Lab data\\outputs\\' + str(row[1]) + '.geojson', 'C:\\Users\\kylek_000\\Documents\\geog458 digital\\Lab data\\outputs\\' + str(row[1]) +'.shp'])
    
    #Table sorted by population 
    top10 = arcpy.Sort_management(table, "C:\\Users\\kylek_000\\Documents\\geog458 digital\\Lab data\\outputs\\", [["Total_POP", "DESCENDING"]])
    
    #Printing the top 10 most populated counties.
    with arcpy.da.SearchCursor(top10, ["CountyName", "Total_POP"]) as sCur:
        i = 0
        for rank in sCur:
            if i < 10:
                print str(rank[0]) + " County has a population of " + str(rank[1]) + "."
                i += 1
                

Call to the method

In [156]:
county_sep(in_state, in_counties)

RuntimeError: A column was specified that does not exist.